In [1]:
#!pip install pyarrow
# !pip install altair
# !pip install geopandas
#!pip install altair-saver --use-feature=2020-resolver
import pandas as pd
import altair as alt
import numpy as np
from datetime import datetime
import sys
import pyarrow
import plotly.express as px
import requests
import geopandas as gpd
import json
import calendar


In [2]:
def calculate_fatality_rate(df, column_cases, column_deaths, new_column_name):
    df[new_column_name] = df[column_deaths].div(df[column_cases].replace(0, np.nan)).fillna(df[column_deaths]) * 100
    return df[new_column_name]
        
def calculate_CFR_for_dataset(df, dataset_name):
    df["num_fatality_rate"] = calculate_fatality_rate(df,"Cases_Total", "Deaths_Total", "num_fatality_rate")
    df["num_fatality_rate_white"] = calculate_fatality_rate(df,"Cases_White", "Deaths_White", "num_fatality_rate_white")
    df["num_fatality_rate_black"] = calculate_fatality_rate(df,"Cases_Black", "Deaths_Black", "num_fatality_rate_black")
    df["num_fatality_rate_latinx"] = calculate_fatality_rate(df,"Cases_Latinx", "Deaths_Latinx", "num_fatality_rate_latinx")
    df["num_fatality_rate_other"] = calculate_fatality_rate(df,"Cases_Other", "Deaths_Other", "num_fatality_rate_other")
    df["num_fatality_rate_missing"] = calculate_fatality_rate(df,"Cases_Missing", "Deaths_Missing", "num_fatality_rate_missing")
    if dataset_name == "cdc":
        df["num_fatality_rate_0_17"] = calculate_fatality_rate(df,"Cases_age_0_17", "Deaths_age_0_17", "num_fatality_rate_0_17")
        df["num_fatality_rate_18_49"] = calculate_fatality_rate(df,"Cases_age_18_49", "Deaths_age_18_49", "num_fatality_rate_18_49")
        df["num_fatality_rate_50_64"] = calculate_fatality_rate(df,"Cases_age_50_64", "Deaths_age_50_64", "num_fatality_rate_50_64")
        df["num_fatality_rate_65_"] = calculate_fatality_rate(df,"Cases_age_65_", "Deaths_age_65_", "num_fatality_rate_65_") 
        df["num_fatality_rate_age_Missing"] = calculate_fatality_rate(df,"Cases_age_Missing", "Deaths_age_Missing", "num_fatality_rate_age_Missing") 
        if "Cases_post_filtering" in df.columns:
            df["num_fatality_rate_post_filtering"] = calculate_fatality_rate(df,"Cases_post_filtering", "Deaths_post_filtering", "num_fatality_rate_post_filtering") 

    return df.round(1)

## CTP data

Now we'll turn to the data from the COVID Tracking Project (CTP), downloaded from [their site](https://covidtracking.com/race/dashboard) on 2021-10-31. The CTP data are cumulative counts of cases and deaths. We will work with the counts from the last day in February 2021 (i.e., the last full month of data before the project shut down). we plot the CTP data by month and ethnic group

In [3]:
origin_ctp_df = pd.read_csv('../data/ctp/CRDT Data - CRDT.csv')
origin_ctp_df['Cases_Other'] = origin_ctp_df[['Cases_Asian','Cases_AIAN','Cases_NHPI','Cases_Multiracial','Cases_Other']].sum(axis=1)
origin_ctp_df['Deaths_Other'] = origin_ctp_df[['Deaths_Asian','Deaths_AIAN','Deaths_NHPI','Deaths_Multiracial','Deaths_Other']].sum(axis=1)
origin_ctp_df['Cases_Missing'] = origin_ctp_df.Cases_Total - origin_ctp_df[['Cases_White','Cases_Black','Cases_Latinx','Cases_Other']].sum(axis=1)
origin_ctp_df['Deaths_Missing'] = origin_ctp_df.Deaths_Total - origin_ctp_df[['Deaths_White','Deaths_Black','Deaths_Latinx','Deaths_Other']].sum(axis=1)
ctp_df = origin_ctp_df.loc[origin_ctp_df.Date==20210228,['Date','State','Cases_Total','Deaths_Total','Cases_White','Deaths_White','Cases_Black',
                               'Deaths_Black','Cases_Latinx','Deaths_Latinx','Cases_Other','Deaths_Other','Cases_Missing',
                               'Deaths_Missing']]

ctp_df.head()

,Date,State,Cases_Total,Deaths_Total,Cases_White,Deaths_White,Cases_Black,Deaths_Black,Cases_Latinx,Deaths_Latinx,Cases_Other,Deaths_Other,Cases_Missing,Deaths_Missing
112,20210228,AK,58293.0,290.0,17991.0,122.0,1481.0,9.0,NaN,NaN,27442.0,154.0,11379.0,5.0
113,20210228,AL,493252.0,9929.0,158420.0,4625.0,81782.0,2168.0,NaN,NaN,40046.0,318.0,213004.0,2818.0
114,20210228,AR,322415.0,5243.0,205690.0,4113.0,50508.0,773.0,NaN,NaN,25410.0,321.0,40807.0,36.0
115,20210228,AS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN
116,20210228,AZ,816782.0,15980.0,304962.0,7874.0,25464.0,423.0,242379.0,4587.0,98718.0,2002.0,145259.0,1094.0


In [4]:
Table_ctp_Data = ctp_df.drop(columns = ["Date", "State"],axis=1).sum(axis=0).to_frame().apply(lambda x: '%.f' % x, axis=1).to_frame().transpose().apply(pd.to_numeric)

## exculding states
def exclude_state(races, origin_df, total_df):
    for race in races:
        states = origin_df.loc[origin_df["Cases_"+ race].isnull() | origin_df["Deaths_"+ race].isnull()]["State"]
        states_data = origin_df.loc[~origin_df.State.isin(states)]
        total_df["Cases_" +race] = states_data.drop(columns = ["Date", "State"],axis=1).sum(axis=0).astype(int)["Cases_"+ race]
        total_df["Deaths_" +race] = states_data.drop(columns = ["Date", "State"],axis=1).sum(axis=0).astype(int)["Deaths_"+ race]
    return total_df

Table_ctp_Data = exclude_state(["White" , "Black" , "Latinx"],ctp_df,Table_ctp_Data)
Table_ctp_Data = calculate_CFR_for_dataset(Table_ctp_Data, "ctp")

## CDC data

Start with the [CDC Public Use data](https://data.cdc.gov/Case-Surveillance/COVID-19-Case-Surveillance-Public-Use-Data-with-Ge/ynhu-f2s2) downloaded on 2021-10-28, restricted to the period up through February 2021 (i.e., when the COVID Tracking Project stopped operating). Data preprocessing performed by [preprocess.py](../data/cdc/preprocess.py).
Read data and generate CFR by month and ethnic group. We also drop data prior to March 2020, since Februrary 2020 includes no deaths.

In [5]:
cdc_df = pd.read_feather('../data/cdc/preprocessed.lz4')
cdc_df["case_month"] = pd.to_datetime(cdc_df['case_month'], format='%Y-%m') 
cdc_df.head()

,case_month,res_state,age_group,sex,race,ethnicity,death_yn
0,2020-11-01,IN,0 - 17 years,NaN,NaN,NaN,No
1,2020-08-01,MI,18 to 49 years,NaN,NaN,NaN,No
2,2020-07-01,WI,18 to 49 years,NaN,NaN,NaN,Unknown
3,2020-12-01,KY,0 - 17 years,Female,NaN,NaN,NaN
4,2020-12-01,IL,0 - 17 years,Female,Missing,Missing,Missing


In [6]:
## calculate of Total Data for Cases and Deaths for CDC
Table_cdc_Data = pd.DataFrame()
Table_cdc_Data["Cases_Total"] = pd.Series(len(cdc_df.loc[(cdc_df.death_yn == "No") | (cdc_df.death_yn == "Yes")]))
Table_cdc_Data["Deaths_Total"] = pd.Series(len(cdc_df.loc[(cdc_df.death_yn == "Yes")]))

In [7]:
def define_ethgroup(dataframe):
    df = dataframe.copy()
    df['ethgrp'] = (
        np.where((df.race=='White') & (df.ethnicity=='Non-Hispanic/Latino'), 1,
        np.where(df.race=='Black', 2,
        np.where(df.ethnicity=='Hispanic/Latino', 3,
        np.where(df.race.isin(['American Indian/Alaska Native',
                               'Asian',
                               'Multiple/Other',
                               'Native Hawaiian/Other Pacific Islander']), 4,
        5)))).astype('int')
    )
    return df

## calculation of Cases and Deaths for QC filtered total data
states = pd.read_csv("../data/covid-cdc-states.csv")
states_data = cdc_df.loc[cdc_df.res_state.isin(states["state"]) & (cdc_df.case_month<='2021-02-28')]
states_data = define_ethgroup(states_data)


In [8]:
## Overall cases and deaths post filerting 38 states

Cases_QC_Overall = states_data.loc[(states_data.death_yn == "Yes") | (states_data.death_yn == "No")]
Deaths_QC_Overall = states_data.loc[(states_data.death_yn == "Yes")]
Table_cdc_Data["Cases_post_filtering"] = pd.Series(len(Cases_QC_Overall))
Table_cdc_Data["Deaths_post_filtering"] = pd.Series(len(Deaths_QC_Overall))

In [9]:
## Calculate Cases by Ethinicity Group After QC Filtering

def total_cases_by_races(races_dict, df):
    for race,ethgrp in races_dict.items():
        df["Cases_" + race] = len(Cases_QC_Overall.loc[Cases_QC_Overall.ethgrp == ethgrp])
        df["Deaths_" + race] = len(Deaths_QC_Overall.loc[Deaths_QC_Overall.ethgrp == ethgrp ])
    return df

Table_cdc_Data = total_cases_by_races(
    {"White" : 1, "Black" : 2, "Latinx" : 3, "Other" : 4 , "Missing"  : 5 },
    Table_cdc_Data)

In [10]:
## Calculation of Cases and Deaths by Age Group 

age_data = states_data.copy()
age_data.age_group = age_data.age_group.fillna("Missing")
age_deaths = age_data.loc[(age_data.death_yn == "Yes")].drop(columns=["case_month","res_state","race", "ethnicity", "sex", "ethgrp"],axis=1).groupby("age_group").count().reset_index()
age_cases = age_data.loc[(age_data.death_yn == "Yes") | (age_data.death_yn == "No")].drop(columns=["case_month","res_state","race", "ethnicity", "sex","ethgrp"],axis=1).groupby("age_group").count().reset_index()

In [11]:
## Adding values of age_cases and age_deaths into Table_cdc_Data

def total_cases_by_age_group(age_group_dict, df):
    for age_key, age_value in age_group_dict.items():
        df["Cases_age_" + age_key] = int(age_cases.loc[age_cases.age_group == age_value]["death_yn"])
        df["Deaths_age_" + age_key] = int(age_deaths.loc[age_deaths.age_group == age_value]["death_yn"])
    return df

Table_cdc_Data = total_cases_by_age_group(
    {"0_17" : "0 - 17 years", "18_49" : "18 to 49 years", "50_64" : "50 to 64 years", "65_" : "65+ years" , "Missing"  : "Missing" },
    Table_cdc_Data)

In [12]:
## Calculate CFR

Table_cdc_Data = calculate_CFR_for_dataset(Table_cdc_Data, "cdc")

In [13]:
%run table.py

In [14]:
headings = ['Name','Deaths','Cases','Observed CFR (%)']

data = [
        ('CTP', '', '', ''),
        ('-----------------------', '------------------', '------------------', '--------------------'),
        ('Overall',"{:,}".format(Table_ctp_Data['Deaths_Total'][0]), "{:,}".format(Table_ctp_Data['Cases_Total'][0]), "{:,}".format(Table_ctp_Data['num_fatality_rate'][0]) ),
        ('', '', '', ''),
        ('Race/Ethnicity', '', '', ''),
        (' White', "{:,}".format(Table_ctp_Data['Deaths_White'][0]), "{:,}".format(Table_ctp_Data['Cases_White'][0]), "{:,}".format(Table_ctp_Data['num_fatality_rate_white'][0])),
        (' Black', "{:,}".format(Table_ctp_Data['Deaths_Black'][0]), "{:,}".format(Table_ctp_Data['Cases_Black'][0]), "{:,}".format(Table_ctp_Data['num_fatality_rate_black'][0])),
        (' Latinx', "{:,}".format(Table_ctp_Data['Deaths_Latinx'][0]), "{:,}".format(Table_ctp_Data['Cases_Latinx'][0]), "{:,}".format(Table_ctp_Data['num_fatality_rate_latinx'][0])),
        (' Others', "{:,}".format(Table_ctp_Data['Deaths_Other'][0]), "{:,}".format(Table_ctp_Data['Cases_Other'][0]), "{:,}".format(Table_ctp_Data['num_fatality_rate_other'][0])),
        (' Missing', "{:,}".format(Table_ctp_Data['Deaths_Missing'][0]), "{:,}".format(Table_ctp_Data['Cases_Missing'][0]), "{:,}".format(Table_ctp_Data['num_fatality_rate_missing'][0])),
        ('-----------------------', '------------------', '------------------', '--------------------'),
        ('CDC', '', '', ''),
        ('-----------------------', '------------------', '------------------', '--------------------'),
        ('Overall',"{:,}".format(Table_cdc_Data['Deaths_Total'][0]), "{:,}".format(Table_cdc_Data['Cases_Total'][0]), "{:,}".format(Table_cdc_Data['num_fatality_rate'][0]) ),
        ('Post-filtering',"{:,}".format(Table_cdc_Data['Deaths_post_filtering'][0]), "{:,}".format(Table_cdc_Data['Cases_post_filtering'][0]), "{:,}".format(Table_cdc_Data['num_fatality_rate_post_filtering'][0]) ),
        ('', '', '', ''),
        ('Age Group', '', '', ''),
        (' < 18', "{:,}".format(Table_cdc_Data['Deaths_age_0_17'][0]), "{:,}".format(Table_cdc_Data['Cases_age_0_17'][0]), "{:,}".format(Table_cdc_Data['num_fatality_rate_0_17'][0])),
        (' 18-49', "{:,}".format(Table_cdc_Data['Deaths_age_18_49'][0]), "{:,}".format(Table_cdc_Data['Cases_age_18_49'][0]), "{:,}".format(Table_cdc_Data['num_fatality_rate_18_49'][0])),
        (' 50-64', "{:,}".format(Table_cdc_Data['Deaths_age_50_64'][0]), "{:,}".format(Table_cdc_Data['Cases_age_50_64'][0]), "{:,}".format(Table_cdc_Data['num_fatality_rate_50_64'][0])),
        (' 65+', "{:,}".format(Table_cdc_Data['Deaths_age_65_'][0]), "{:,}".format(Table_cdc_Data['Cases_age_65_'][0]), "{:,}".format(Table_cdc_Data['num_fatality_rate_65_'][0])), 
        (' Missing', "{:,}".format(Table_cdc_Data['Deaths_age_Missing'][0]), "{:,}".format(Table_cdc_Data['Cases_age_Missing'][0]), "{:,}".format(Table_cdc_Data['num_fatality_rate_age_Missing'][0])), 
        ('', '', '', ''),
        ('Race/Ethnicity', '', '', ''),
        (' White', "{:,}".format(Table_cdc_Data['Deaths_White'][0]), "{:,}".format(Table_cdc_Data['Cases_White'][0]), "{:,}".format(Table_cdc_Data['num_fatality_rate_white'][0])),
        (' Black', "{:,}".format(Table_cdc_Data['Deaths_Black'][0]), "{:,}".format(Table_cdc_Data['Cases_Black'][0]), "{:,}".format(Table_cdc_Data['num_fatality_rate_black'][0])),
        (' Latinx', "{:,}".format(Table_cdc_Data['Deaths_Latinx'][0]), "{:,}".format(Table_cdc_Data['Cases_Latinx'][0]), "{:,}".format(Table_cdc_Data['num_fatality_rate_latinx'][0])),
        (' Others', "{:,}".format(Table_cdc_Data['Deaths_Other'][0]), "{:,}".format(Table_cdc_Data['Cases_Other'][0]), "{:,}".format(Table_cdc_Data['num_fatality_rate_other'][0])),
        (' Missing', "{:,}".format(Table_cdc_Data['Deaths_Missing'][0]), "{:,}".format(Table_cdc_Data['Cases_Missing'][0]), "{:,}".format(Table_cdc_Data['num_fatality_rate_missing'][0])),
        ('-----------------------', '------------------', '------------------', '--------------------'),

]

fields = [0,1,2,3]

align = [('^', '<'), ('^', '<'), ('^', '<'), ('^', '<'), ]

table(sys.stdout, data, fields, headings, align)

|          Name           |       Deaths       |       Cases        |   Observed CFR (%)   |
| ----------------------- | ------------------ | ------------------ | -------------------- |
| CTP                     |                    |                    |                      |
| ----------------------- | ------------------ | ------------------ | -------------------- |
| Overall                 | 512,627            | 28,443,555         | 1.8                  |
|                         |                    |                    |                      |
| Race/Ethnicity          |                    |                    |                      |
|  White                  | 268,373            | 10,309,363         | 2.6                  |
|  Black                  | 61,989             | 2,359,473          | 2.6                  |
|  Latinx                 | 71,844             | 3,666,785          | 2.0                  |
|  Others                 | 39,403             | 2,539,435          | 

## CDC Race Data

Read data and generate CFR by month and ethnic group. We also drop data prior to March 2020, since Februrary 2020 includes no deaths. Plot CFR by month and ethnic group.

In [15]:
def calculate_figure1_race_data(races_dict):
    df = pd.DataFrame()
    for race,ethgrp in races_dict.items():
        cases_df = Cases_QC_Overall.loc[Cases_QC_Overall.ethgrp == ethgrp ]
        deaths_df = Deaths_QC_Overall.loc[Deaths_QC_Overall.ethgrp == ethgrp ]
        internal_df = cases_df.groupby("case_month")["death_yn"].count().to_frame(
        ).join(deaths_df.groupby("case_month")["death_yn"].count().to_frame(), 
               lsuffix="_case", rsuffix="_death").reset_index()
        internal_df["CFR"] = calculate_fatality_rate(internal_df, "death_yn_case", "death_yn_death", "CFR")
        internal_df["CFR"] = internal_df["CFR"].div(100)
        internal_df = internal_df.loc[internal_df.case_month >= "2020-03-01"]
        internal_df["Race"] = race
        df = internal_df if df.empty else pd.concat([df,internal_df]).reset_index().drop(columns=["index"])
    return df
    
figure1_dataset_cdc = calculate_figure1_race_data(
    {"White" : 1, "Black" : 2, "Latinx" : 3})

fig1 = alt.Chart(figure1_dataset_cdc).mark_line().encode(
    alt.X('yearmonth(case_month):T',
          axis=alt.Axis(title='Month', grid=False, labelFlush=False)),
    alt.Y('CFR:Q',
          axis=alt.Axis(title='CFR (%)', format='%'),
          scale=alt.Scale(type='log')),
    color=alt.Color('Race:N',
                    legend=alt.Legend(title=['Racial and', 'ethnic group']))
).properties(
    title='CDC Data'
)

fig1

alt.Chart(...)

## Figure 3

Plot state-specific CFRs, separately by ethnic group and age group.

In [16]:
## Calculating Data for Figure 3 calculating state-specific CFRs, separately by ethnic group and age group

figure3_data = states_data.loc[
    (states_data.case_month >= "2020-03-01")
    ].reset_index().drop(columns=["index"]).groupby(["res_state","age_group","ethgrp","death_yn"])["case_month"].count().reset_index()

## getting Cases and deaths data from the above filtered data to calculate CFR

figure3_data_cases = figure3_data.loc[
    (figure3_data.death_yn == "Yes") | (figure3_data.death_yn == "No")
    ].groupby(["res_state","age_group","ethgrp"]).sum().fillna(0).astype(int).rename(columns={"case_month" : "case_count"})

figure3_data_deaths = figure3_data.loc[
    (figure3_data.death_yn == "Yes")
    ].groupby(["res_state","age_group","ethgrp"]).sum().fillna(0).astype(int).rename(columns={"case_month" : "death_count"})

figure3_data_cases_deaths = figure3_data_cases.join(figure3_data_deaths,lsuffix="_cases", rsuffix="_deaths")
figure3_data_cases_deaths["CFR"] = calculate_fatality_rate(figure3_data_cases_deaths, "case_count", "death_count", "CFR") 
figure3_data_cases_deaths = figure3_data_cases_deaths.reset_index()

## we are only projecting data from age_group "18 to 49 years","50 to 64 years","65+ years" and
##  race Black, White and Latinx

figure3_data_cases_deaths = figure3_data_cases_deaths.loc[
    (figure3_data_cases_deaths.age_group.isin(["18 to 49 years","50 to 64 years","65+ years"])) 
    & (figure3_data_cases_deaths.ethgrp < 4) ]

figure3_data_cases_deaths = figure3_data_cases_deaths.replace({"ethgrp": { 1 : "White" ,  2 : "Black" ,  3: "Latinx" }})

In [17]:
## plotting figure 3

alt.data_transformers.enable('default', max_rows=None)
fig3 = alt.Chart(figure3_data_cases_deaths).transform_filter(
    alt.datum.CFR > 0  
).mark_circle(
    opacity=0.8,
    stroke='black',
    strokeWidth=1
).encode(
    alt.X('CFR:Q', axis=alt.Axis(title='CFR (%)', format='%'),
          scale=alt.Scale(type='log')),
    alt.Y('ethgrp:N', title=None),
    alt.Color('ethgrp:N', legend=alt.Legend(title=['Racial and', 'ethnic group'])),
    alt.Size('case_count:Q',
        legend=alt.Legend(title='Number of cases'))
).facet(
    row=alt.Row('age_group:N', title=None, header=alt.Header(labelOrient='top'))
).resolve_scale(
    x='independent'
).properties(
    title='Figure 3. City of chicgao CFRs, separately by age group and racial and ethnic group'
).configure_title(
    anchor='start'
)
fig3


alt.FacetChart(...)